In [157]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
pd.set_option('display.max_rows', 10) 

In [158]:
# Load the CSV data into a DataFrame
df = pd.read_csv('LaLigaHistory.csv', index_col= 0)

# Convert the date column to datetime objects for easier date manipulation
df['date_GMT'] = pd.to_datetime(df['date_GMT'], format = ('%b %d %Y - %I:%M%p'))

In [159]:
# Display the DataFrame and its data types for verification
#print(df.dtypes)

In [160]:
# Convert categorical text data to numerical codes for model training

df["home_team_code"] = df["home_team_name"].astype("category").cat.codes
df["away_team_code"] = df["away_team_name"].astype("category").cat.codes
df["stadium_code"] = df["stadium_name"].astype("category").cat.codes

# Extract the day of the week from the date to use as a feature

df["day_of_week"] = df["date_GMT"].dt.day_of_week

# Map match results to numerical values: 1 for Win (W), 0 for Loss (L), 2 for Draw (D)

df["target"] = df["Result"].map({"W": 1, "L": 0, "D": 2})

# Display the first few rows to ensure correct mapping and transformations

#df.head()

In [161]:
train = df[df["date_GMT"] < '2023-08-05']
test = df[(df["date_GMT"] >= '2023-08-05') & (df["date_GMT"] < '2024-04-01')]

# Initialize the RandomForestClassifier with specified hyperparameters
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state= 1)

# Define the predictor variables/features to use for training the model
predictor = ["home_team_code","away_team_code", "stadium_code", 
             "day_of_week","Game Week","time"]

rf.fit(train[predictor], train["target"])
preds = rf.predict(test[predictor])
error = accuracy_score(test["target"], preds)

In [162]:
# Create a DataFrame to compare actual vs predicted results
combined = pd.DataFrame(dict(actual=test["target"], predicted = preds))

# Display a crosstabulation of actual vs predicted results for better analysis
pd.crosstab(index = combined["actual"], columns=combined["predicted"])

predicted,0,1,2
actual,,,
0,25,40,12
1,11,97,25
2,18,61,9


In [163]:
grouped_home_team_df = df.groupby("home_team_name")
group_home = grouped_home_team_df.get_group("Real Madrid")

In [164]:
def rolling_avg(group_home, cols, new_cols):
    group_home = group_home.sort_values("date_GMT")
    rolling_stats_home = group_home[cols].rolling(20, closed='left').mean()
    group_home[new_cols] = rolling_stats_home
    group_home = group_home.dropna(subset=new_cols)
    return group_home

In [165]:
cols = ["home_team_goal_count", 
        "home_team_shots",
        "home_team_shots_on_target", ]
new_cols = [f"{c}_rolling" for c in cols]

In [166]:
rolling_avg(group_home, cols, new_cols)

,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,...,Result,time,home_team_code,away_team_code,stadium_code,day_of_week,target,home_team_goal_count_rolling,home_team_shots_rolling,home_team_shots_on_target_rolling
timestamp,,,,,,,,,,,,,,,,,,,,,
1602952200,2020-10-17 16:30:00,complete,NaN,Real Madrid,Cadiz,Santiago Jaime Latre,6,3.0,3.00,2.21,...,L,16,19,5,7,5,0,1.65,14.35,7.30
1604149200,2020-10-31 13:00:00,complete,0.0,Real Madrid,SD Huesca,Pablo González Fuertes,8,1.5,0.75,2.21,...,W,13,19,23,7,5,1,1.55,14.60,7.20
1606593600,2020-11-28 20:00:00,complete,0.0,Real Madrid,Deportivo Alavés,Adrián Cordero Vega,11,2.0,1.00,2.21,...,L,20,19,7,7,5,0,1.55,14.55,7.10
1607803200,2020-12-12 20:00:00,complete,0.0,Real Madrid,Atletico Madrid,Antonio Miguel Mateu Lahoz,13,1.5,2.50,2.21,...,W,20,19,2,7,5,1,1.55,14.75,7.05
1608066000,2020-12-15 21:00:00,complete,0.0,Real Madrid,Athletic Club Bilbao,Jesús Gil Manzano,19,1.8,0.71,2.21,...,W,21,19,1,7,1,1,1.65,14.60,6.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743890400,2025-04-05 22:00:00,incomplete,NaN,Real Madrid,Valencia CF,NaN,30,0.0,0.00,0.00,...,D,22,19,26,21,5,2,0.85,3.80,2.15
1745100000,2025-04-19 22:00:00,incomplete,NaN,Real Madrid,Athletic Club Bilbao,NaN,32,0.0,0.00,0.00,...,D,22,19,1,21,5,2,0.65,2.65,1.40
1746309600,2025-05-03 22:00:00,incomplete,NaN,Real Madrid,Celta de Vigo,NaN,34,0.0,0.00,0.00,...,D,22,19,6,21,5,2,0.55,2.20,1.10


In [167]:
df_rolling = df.groupby("home_team_name").apply(lambda x: rolling_avg(x,cols,new_cols))
#df_rolling

C:\Users\munoz\AppData\Local\Temp\ipykernel_15540\1900319699.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df.groupby("home_team_name").apply(lambda x: rolling_avg(x,cols,new_cols))


In [168]:
df_rolling.droplevel("home_team_name")

,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,...,Result,time,home_team_code,away_team_code,stadium_code,day_of_week,target,home_team_goal_count_rolling,home_team_shots_rolling,home_team_shots_on_target_rolling
timestamp,,,,,,,,,,,,,,,,,,,,,
1692466200,2023-08-19 17:30:00,complete,NaN,Almería,Real Madrid,José María Sánchez Martínez,2,0.00,3.0,0.58,...,L,17,0,19,31,5,0,1.45,13.25,5.30
1693598400,2023-09-01 20:00:00,complete,NaN,Almería,Celta de Vigo,Ricardo De Burgos Bengoetxea,4,0.00,1.0,0.58,...,L,20,0,6,31,4,0,1.45,13.55,5.40
1695495600,2023-09-23 19:00:00,complete,NaN,Almería,Valencia CF,Mateo Busquets Ferrer,6,0.00,1.5,0.58,...,D,19,0,26,31,5,2,1.45,13.55,5.40
1696161600,2023-10-01 12:00:00,complete,NaN,Almería,Granada CF,Pablo González Fuertes,8,0.25,0.0,0.58,...,D,12,0,12,31,6,2,1.55,13.95,5.80
1698494400,2023-10-28 12:00:00,complete,NaN,Almería,UD Las Palmas,Francisco José Hernández Maeso,11,0.40,0.6,0.58,...,L,12,0,25,31,5,0,1.55,14.25,5.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743890400,2025-04-05 22:00:00,incomplete,NaN,Villarreal,Athletic Club Bilbao,NaN,30,1.00,0.0,1.00,...,D,22,27,1,27,5,2,0.95,4.30,1.85
1745100000,2025-04-19 22:00:00,incomplete,NaN,Villarreal,Real Sociedad,NaN,32,1.00,0.0,1.00,...,D,22,27,20,27,5,2,0.70,3.55,1.25
1746309600,2025-05-03 22:00:00,incomplete,NaN,Villarreal,CA Osasuna,NaN,34,1.00,0.0,1.00,...,D,22,27,4,27,5,2,0.65,2.85,0.80


In [169]:
df_rolling.index = range(df_rolling.shape[0])
#df_rolling

In [170]:
from sklearn.metrics import precision_score

def future(data, predictor):
    train = data[data["date_GMT"] < '2024-04-01']
    test = data[data["date_GMT"] > '2024-04-01']
    rf.fit(train[predictor], train["target"])
    preds = rf.predict(test[predictor])
    combined = pd.DataFrame(dict(actual = test["target"], predictor = preds), index = test.index)
    precision = precision_score(test["target"], preds, average='weighted')

    return  precision, combined



In [171]:
precision, combined = future(df_rolling, predictor + new_cols) 
precision

np.float64(0.8316516482792844)

In [172]:
#combined

In [173]:
combined  = combined.merge(df_rolling[["date_GMT", "home_team_name", "away_team_name", "Result"]], left_index = True, right_index = True)
combined

,actual,predictor,date_GMT,home_team_name,away_team_name,Result
14,0,2,2024-04-21 14:15:00,Almería,Villarreal,L
15,0,0,2024-04-27 14:15:00,Almería,Getafe CF,L
16,0,2,2024-05-16 19:30:00,Almería,FC Barcelona,L
17,1,2,2024-05-25 16:30:00,Almería,Cadiz,W
89,2,1,2024-04-14 16:30:00,Athletic Club Bilbao,Villarreal,D
...,...,...,...,...,...,...
1717,2,2,2025-04-05 22:00:00,Villarreal,Athletic Club Bilbao,D
1718,2,2,2025-04-19 22:00:00,Villarreal,Real Sociedad,D
1719,2,2,2025-05-03 22:00:00,Villarreal,CA Osasuna,D
1720,2,2,2025-05-13 22:00:00,Villarreal,Leganés,D


In [174]:

def calculate_final_standings(combined):
    standings = {}


    result_mapping = {1: 'Win', 0: 'Loss', 2: 'Draw'}

    for index, row in combined.iterrows():
        home_team = row['home_team_name']
        away_team = row['away_team_name']
        result = row['actual'] 
        
        if home_team not in standings:
            standings[home_team] = 0
        if away_team not in standings:
            standings[away_team] = 0


        if result == 1:  # Home win
            standings[home_team] += 3
        elif result == 0:  # Away win
            standings[away_team] += 3
        elif result == 2:  # Draw
            standings[home_team] += 1
            standings[away_team] += 1



    standings_df = pd.DataFrame(list(standings.items()), columns=['Team', 'Points'])

    final_df = standings_df.sort_values(by='Points', ascending=False).reset_index(drop=True)




    print("\nFinal League Standings:")
    print(final_df)


calculate_final_standings(combined)

    


Final League Standings:
               Team  Points
0   Atletico Madrid      59
1       Real Madrid      58
2      FC Barcelona      58
3        Villarreal      55
4     Celta de Vigo      55
..              ...     ...
18     RCD Espanyol      37
19    UD Las Palmas      37
20          Almería       8
21            Cadiz       8
22       Granada CF       7

[23 rows x 2 columns]
